<a href="https://colab.research.google.com/github/hargurjeet/LJMU_Thesis/blob/main/simpler_embedding_for_all_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hargurjeet/LJMU_Thesis.git

Cloning into 'LJMU_Thesis'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 108 (delta 57), reused 9 (delta 3), pack-reused 0
Receiving objects: 100% (108/108), 464.56 KiB | 4.84 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [2]:
!pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [19]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score,classification_report
import numpy as np
from sentence_transformers import SentenceTransformer
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
# Load the Sentence Transformer model
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Open AI Chatgpt

In [5]:
df_open_ai = pd.read_csv("/content/LJMU_Thesis/Processed_output/openai_generated_output.csv")
df_open_ai.head()

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target,TextualData
0,34,No,Middle Income,6,No,Yes,0,The customer is 34 years old and has indicated...
1,34,Yes,Low Income,5,Yes,No,1,The customer is aged 34 and is a frequent flye...
2,37,No,Middle Income,3,Yes,No,0,The customer is 37 years old and is not a freq...
3,30,No,Middle Income,2,No,No,0,The customer is aged 30 and is not a frequent ...
4,30,No,Low Income,1,No,No,0,"The customer is 30 years old, does not frequen..."


In [6]:
def get_embedding(text):
    # Generate and return the embedding using the Sentence Transformer model
    embedding = model.encode(text, convert_to_tensor=True)  # Optional: convert_to_tensor=True returns a PyTorch tensor
    return embedding

df_open_ai['Embedding'] = df_open_ai['TextualData'].apply(get_embedding)

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming dummy_df is already populated with TextualData and Embedding columns

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    df_open_ai['Embedding'].tolist(),
    df_open_ai['Target'],
    test_size=0.2,
    random_state=42
)

# Standardize the embeddings
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Dimensionality Reduction with PCA
# pca = PCA(n_components=min(len(X_train), X_train.shape[1]))
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_pca, y_train)

# Make predictions and evaluate the model
y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.83


In [8]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.82      1.00      0.90       153
           1       1.00      0.13      0.23        38

    accuracy                           0.83       191
   macro avg       0.91      0.57      0.57       191
weighted avg       0.86      0.83      0.77       191



Mixtral

In [9]:
df_mixtral = pd.read_csv("/content/LJMU_Thesis/Processed_output/mixtral_generated_output.csv")
df_mixtral.head()

,Unnamed: 0.1,Unnamed: 0,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target,GeneratedText
0,0,0,34,No,Middle Income,6,No,Yes,0,The customer is 34 years old and does not have...
1,1,1,34,Yes,Low Income,5,Yes,No,1,The customer's age is 34. This is a significan...
2,2,2,37,No,Middle Income,3,Yes,No,0,The customer's age is 37. This is a significan...
3,3,3,30,No,Middle Income,2,No,No,0,The customer is a 30-year-old individual who d...
4,4,4,30,No,Low Income,1,No,No,0,The customer's age is 30. This could indicate ...


In [ ]:
df_mixtral['Embedding'] = df_mixtral['GeneratedText'].apply(get_embedding)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    df_mixtral['Embedding'].tolist(),
    df_mixtral['Target'],
    test_size=0.2,
    random_state=42
)

# Standardize the embeddings
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Dimensionality Reduction with PCA
# pca = PCA(n_components=min(len(X_train), X_train.shape[1]))
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_pca, y_train)

# Make predictions and evaluate the model
y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print(report)

Accuracy: 0.81
              precision    recall  f1-score   support

           0       0.82      0.99      0.89       153
           1       0.67      0.11      0.18        38

    accuracy                           0.81       191
   macro avg       0.74      0.55      0.54       191
weighted avg       0.79      0.81      0.75       191



## llama

In [13]:
df_llama = pd.read_csv("/content/LJMU_Thesis/Processed_output/llama_generated_output.csv")
df_llama.head()

,Age,FrequentFlyer,AnnualIncomeClass,ServicesOpted,AccountSyncedToSocialMedia,BookedHotelOrNot,Target,GeneratedText
0,37,No,Low Income,1,No,Yes,0,The customer in question is a 37-year-old indi...
1,30,No,Middle Income,4,No,No,0,This customer is a 30-year-old individual with...
2,31,No,Middle Income,1,Yes,Yes,0,This customer is a 31-year-old individual with...
3,30,Yes,Low Income,1,Yes,No,1,This customer is a 30-year-old individual who ...
4,30,No Record,Middle Income,3,No,No,0,"This customer is 30 years old, indicating that..."


In [18]:
df_llama['Embedding'] = df_llama['GeneratedText'].apply(get_embedding)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    df_llama['Embedding'].tolist(),
    df_llama['Target'],
    test_size=0.2,
    random_state=42
)

# Standardize the embeddings
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Dimensionality Reduction with PCA
# pca = PCA(n_components=min(len(X_train), X_train.shape[1]))
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train a RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_pca, y_train)

# Make predictions and evaluate the model
y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print(report)

Accuracy: 0.79
              precision    recall  f1-score   support

           0       0.79      0.99      0.88       150
           1       0.67      0.05      0.09        41

    accuracy                           0.79       191
   macro avg       0.73      0.52      0.49       191
weighted avg       0.77      0.79      0.71       191



## Experimenting with dimensions